# Fantasy Football Analysis

In [3]:
# Import neccessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import dash as dash
from dash import dash_table
from dash import dcc, html
from dash.dependencies import Input, Output

## Web Scraping
We're going to use Beautiful Soup to scrape the website. It loops through different drop down arrows, pulling the necessary data.

In [4]:
# Define base URL
base_url = "https://www.footballguys.com/playerhistoricalstats?pos=rb&yr={year}&startwk=1&stopwk=17&profile=p"

In [6]:
# Specify the years we want to scrape
years = range(1996, 2025)
positions = ['qb', 'rb', 'wr', 'te']

In [14]:
# Initialize a dictionary to store DataFrames for each position
position_data = {}

# Loop through each position
for position in positions:
    position_stats = []

    # Loop through each year
    for year in years:
        url = base_url.format(position=position, year=year)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # Find the table and extract headers dynamically
        table = soup.find("table", class_="datasmall table")
        if table:
            # Extract headers by finding all <th> elements
            headers = [header.text.strip() for header in table.find_all("th")]
            headers = ["Year"] + headers  # Add 'Year' as the first column

            # Extract rows
            rows = table.find_all("tr")
            for row in rows:
                cells = row.find_all("td")
                # Only process rows that have data (ignore rows with no data)
                if len(cells) > 0:
                    row_data = {
                        "Year": year,
                        **{headers[i + 1]: cell.text.strip() for i, cell in enumerate(cells)}
                    }
                    position_stats.append(row_data)

    # Convert the position's stats to a DataFrame
    position_df = pd.DataFrame(position_stats)
    position_data[position] = position_df

   Year Rank                  Name   Age   Exp   G  Cmp  Att   Cm%   PYd  \
0  1996    1        Brett Favre GB  27.0   6.0  16  325  543  59.9  3895   
1  1996    2  Vinny Testaverde BAL  33.0  10.0  16  325  549  59.2  4177   
2  1996    3      Mark Brunell JAX  26.0   3.0  16  353  557  63.4  4367   
3  1996    4        John Elway DEN  36.0  14.0  15  287  466  61.6  3328   
4  1996    5        Jeff Blake CIN  26.0   5.0  16  308  549  56.1  3624   

  Y/Att PTD Int Rsh RshYd RshTD  FP/G FantPt  
0  7.17  39  14  49   136     2  19.3  309.4  
1  7.61  33  19  34   188     2  18.2  291.9  
2  7.84  19  18  79   400     3  17.0  272.7  
3  7.14  26  14  50   249     4  17.2  258.0  
4  6.60  24  14  73   313     2  16.0  256.3  


In [13]:
# Create DataFrame
fantasy_df = pd.DataFrame(all_data, columns=headers)
fantasy_df.head(5)

,Year,Rank,Name,Age,Exp,G,Rec,RecYd,Y/Rec,RecTD,FP/G,FantPt
0,1996,1,Brett Favre GB,27.0,6.0,16,NaN,NaN,NaN,NaN,19.3,309.4
1,1996,2,Vinny Testaverde BAL,33.0,10.0,16,NaN,NaN,NaN,NaN,18.2,291.9
2,1996,3,Mark Brunell JAX,26.0,3.0,16,NaN,NaN,NaN,NaN,17.0,272.7
3,1996,4,John Elway DEN,36.0,14.0,15,NaN,NaN,NaN,NaN,17.2,258.0
4,1996,5,Jeff Blake CIN,26.0,5.0,16,NaN,NaN,NaN,NaN,16.0,256.3


## Tidying the Data

In [9]:
# Drop empty rows
fantasy_df = fantasy_df.dropna(thresh=7)


# Change to correct data types
fantasy_df[['Rank', 'FP/G', 'FantPt', 'Age', 'Exp', 'G']] = fantasy_df[['Rank', 'FP/G', 'FantPt', 'Age', 'Exp', 'G']].astype(float)

# Rename columns
fantasy_df.columns = ['Year', 'Rank', 'Name', 'Age', 'Experience', 'Games_Played', 'Points_Per_Game', 'Total_Points']

# Create 'Team' column
fantasy_df['Team'] = fantasy_df['Name'].apply(lambda x: x.split()[-1] if x.split()[-1].isupper() else '')

fantasy_df.head(5)

KeyError: "['Rsh', 'RshYd', 'Y/Rsh', 'RshTD'] not found in axis"

In [8]:
# Total number of rows
total_rows = len(fantasy_df)
print(f'Total Rows: {total_rows}')

# Total number of running backs
unique_rbs = fantasy_df['Name'].nunique()
print(f'Unique RBs: {unique_rbs}')

Total Rows: 3963
Unique RBs: 1803


I don't want the data to be scewed by the many running backs that barely ever played during their career. I only want to look at the running backs that have had an impact. However, I want to see if they've declined over time. In Fantasy Football, you usually have to be a top 30 running back to be useful. So, I'm only going to keep the running backs that at one point in time were in the top 30.

In [9]:
# Identify running backs who have ever ranked 30 or less
rbs_with_top_rank = fantasy_df.groupby('Name')['Rank'].min()
top_rbs = rbs_with_top_rank[rbs_with_top_rank <= 30].index

# Filter the original DataFrame to keep only rows for these top running backs
fantasy_df_filtered = fantasy_df[fantasy_df['Name'].isin(top_rbs)]

fantasy_df_filtered.head(5)


,Year,Rank,Name,Age,Experience,Games_Played,Points_Per_Game,Total_Points,Team
1,2000,1.0,Marshall Faulk STL,27.0,7.0,14.0,32.6,455.9,STL
2,2000,2.0,Edgerrin James IND,22.0,2.0,16.0,25.1,401.3,IND
3,2000,3.0,Eddie George TEN,27.0,5.0,16.0,21.4,342.0,TEN
4,2000,4.0,Ahman Green GB,23.0,3.0,16.0,20.3,324.4,GB
5,2000,5.0,Charlie Garner SF,28.0,7.0,16.0,19.2,306.9,SF


At this point, this is what our final dataset should look like, so I'm good to upload it as a CSV and continue doing EDA.

In [10]:
fantasy_df_filtered.to_csv('Data/fantasy.csv', index=False)

## Exploratory Data Analysis

In [11]:
fantasy_df_filtered.describe()


,Year,Rank,Age,Experience,Games_Played,Points_Per_Game,Total_Points
count,1316.000000,1316.000000,1316.000000,1316.000000,1316.000000,1316.000000,1316.000000
mean,2011.874620,36.427052,25.905015,4.534954,12.771277,11.950000,159.286398
std,6.952982,31.397367,2.934666,2.842560,3.857949,5.445604,90.088398
min,2000.000000,1.000000,21.000000,1.000000,1.000000,-0.200000,-0.200000
25%,2006.000000,14.000000,24.000000,2.000000,11.000000,8.100000,91.250000
50%,2012.000000,27.000000,25.000000,4.000000,14.000000,11.800000,157.250000
75%,2018.000000,50.000000,28.000000,6.000000,16.000000,15.400000,214.925000
max,2024.000000,160.000000,36.000000,15.000000,16.000000,32.600000,483.100000


This is a Dash app that I can use to quickly see different years and the ages over the years. I used generative AI to help me generate it. It's really cool and I would love to include it in the blog, but I looked into it and it's really hard to deploy it online.

In [14]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.H1(
                    'Top 10 Running Backs by Total Points Over the Years',
                    style={
                        'textAlign': 'center',
                        'color': '#2c3e50',
                        'fontFamily': 'Arial, sans-serif',
                        'marginTop': '20px'
                    }
                ),
                html.Div(
                    children=[
                        dcc.Dropdown(
                            id='year-dropdown',
                            options=[
                                {'label': str(year), 'value': year} 
                                for year in fantasy_df_filtered['Year'].unique()
                            ],
                            value=2023,  # Default selected year
                            style={
                                'width': '50%',
                                'margin': 'auto',
                                'padding': '10px',
                                'fontSize': '16px',
                                'fontFamily': 'Arial, sans-serif'
                            }
                        )
                    ],
                    style={
                        'display': 'flex',
                        'justifyContent': 'center',
                        'marginBottom': '20px'
                    }
                ),
                dash_table.DataTable(
                    id='running-backs-table',
                    columns=[
                        {'name': 'Rank', 'id': 'Rank'},
                        {'name': 'Name', 'id': 'Name'},
                        {'name': 'Total Points', 'id': 'Total_Points'},
                        {'name': 'Age', 'id': 'Age'}
                    ],
                    style_table={
                        'height': '400px',
                        'overflowY': 'auto',
                        'borderRadius': '8px',
                        'boxShadow': '0px 0px 15px rgba(0, 0, 0, 0.1)',
                        'backgroundColor': '#ffffff',
                        'border': '1px solid #ddd',
                        'margin': 'auto',
                        'padding': '20px'
                    },
                    style_header={
                        'backgroundColor': '#2980b9',
                        'color': 'white',
                        'fontWeight': 'bold',
                        'textAlign': 'center',
                        'fontFamily': 'Arial, sans-serif'
                    },
                    style_cell={
                        'textAlign': 'center',
                        'fontFamily': 'Arial, sans-serif',
                        'padding': '10px'
                    },
                    style_data={
                        'backgroundColor': '#f9f9f9',
                        'color': '#2c3e50'
                    },
                    style_data_conditional=[
                        {
                            'if': {
                                'row_index': 'odd'
                            },
                            'style': {
                                'backgroundColor': '#ecf0f1'
                            }
                        }
                    ]
                ),
            ],
            style={
                'maxWidth': '1200px',
                'margin': 'auto',
                'backgroundColor': '#ecf0f1',
                'padding': '30px',
                'borderRadius': '10px',
                'boxShadow': '0px 0px 20px rgba(0, 0, 0, 0.1)',
            }
        )
    ]
)

# Callback to update the table based on selected year
@app.callback(
    Output('running-backs-table', 'data'),
    [Input('year-dropdown', 'value')]
)
def update_table(selected_year):
    # Filter the DataFrame by the selected year
    filtered_df = fantasy_df_filtered[fantasy_df_filtered['Year'] == selected_year]
    
    # Sort by Total Points in descending order and take top 10
    top_10 = filtered_df.sort_values(by='Total_Points', ascending=False).head(10)
    
    # Convert DataFrame to a dictionary to pass into the table
    return top_10.to_dict('records')

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

Now I want to look at the change in the mean age over the years.

In [ ]:
# Group by 'Year' and calculate the mean age for each year
mean_age_per_year = fantasy_df_filtered.groupby('Year')['Age'].mean().reset_index()

# Rename the columns for clarity
mean_age_per_year.columns = ['Year', 'Mean_Age']

# Display the result
mean_age_per_year.head(25)

,Year,Mean_Age
0,2000,26.000000
1,2001,26.638298
2,2002,26.588235
3,2003,26.745455
4,2004,26.592593
5,2005,26.827586
6,2006,26.377358
7,2007,26.327273
8,2008,25.843750
9,2009,25.790323


I don't see much of a change in the average year. What if I look at the counts of the ages of RBs in the top 30?

In [ ]:
# Step 1: Filter the top 10 players based on 'Rank'
top_30_df = fantasy_df_filtered[fantasy_df_filtered['Rank'] <= 30]

# Step 2: Define the age bins and labels
bins = [20, 23, 26, 29, 100]  # Bins: 21-23, 24-26, 27-29, 30+
labels = ['21-23', '24-26', '27-29', '30+']

# Step 3: Create a new column in the DataFrame for the age bins
top_30_df['Age_Bin'] = pd.cut(top_30_df['Age'], bins=bins, labels=labels, right=True)

# Step 4: Count the occurrences of each age bin by year
age_bin_count = top_30_df.groupby(['Year', 'Age_Bin']).size().reset_index(name='Age_Count')

# Step 5: Calculate the percentage of each age bin within each year
age_bin_count['Percentage'] = age_bin_count.groupby('Year')['Age_Count'].transform(lambda x: x / x.sum() * 100)

# Step 6: Create the stacked histogram with percentage
fig = px.bar(
    age_bin_count,
    x='Year',
    y='Percentage',
    color='Age_Bin',
    title="Age Distribution of Top 10 Running Backs Over the Years (Percentage)",
    labels={"Year": "Year", "Percentage": "Percentage of Players", "Age_Bin": "Age Range"},
    barmode='stack',
    height=600,
    width=900,
)

# Step 7: Adjust layout for better appearance and add space between bars
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Percentage of Players",
    legend_title="Age Range",
    template="plotly_dark",  # Dark theme for better contrast
    title_x=0.5,  # Center the title
    title_y=0.95,  # Move the title up a bit
    plot_bgcolor="#2b2b2b",  # Dark background
    paper_bgcolor="#2b2b2b",  # Dark background for the whole figure
    font=dict(family="Arial", size=14, color="white"),
    bargap=0.2,  # Add space between bars
)

# Step 8: Show the plot
fig.show()

/var/folders/fn/0m355dy57hlgk4fkg7n62xhc0000gp/T/ipykernel_78060/3126758318.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/fn/0m355dy57hlgk4fkg7n62xhc0000gp/T/ipykernel_78060/3126758318.py:12: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [28]:
import plotly.graph_objects as go

# Group the data and calculate the mean for Points_Per_Game and Games_Played
age_means = fantasy_df_filtered.groupby('Age')[['Points_Per_Game', 'Games_Played']].mean().reset_index()

# Create the figure with Plotly
fig = go.Figure()

# Add the Points Per Game line
fig.add_trace(go.Scatter(
    x=age_means['Age'],
    y=age_means['Points_Per_Game'],
    mode='lines+markers',
    name='Points Per Game',
    line=dict(color='skyblue', width=2),
    marker=dict(size=8)
))

# Add the Games Played line
fig.add_trace(go.Scatter(
    x=age_means['Age'],
    y=age_means['Games_Played'],
    mode='lines+markers',
    name='Games Played',
    line=dict(color='lightgreen', width=2, dash='dash'),
    marker=dict(size=8)
))

# Update layout for a dark theme without grid lines
fig.update_layout(
    title='<b>Average Points Per Game and Games Played by Age</b>',  # Bold title using HTML tags
    title_font=dict(size=16, color='white', family='Arial'),
    xaxis=dict(
        title='Age', 
        title_font=dict(size=14, color='white'), 
        tickfont=dict(size=12, color='white'),
        showgrid=False  # Remove grid lines
    ),
    yaxis=dict(
        title='Mean Values', 
        title_font=dict(size=14, color='white'), 
        tickfont=dict(size=12, color='white'),
        showgrid=False  # Remove grid lines
    ),
    plot_bgcolor='black',  # Set plot background color
    paper_bgcolor='rgb(30, 30, 30)',  # Set paper background color
    font=dict(color='white'),
    legend=dict(title='Metrics', title_font=dict(size=13, color='white'), font=dict(size=11, color='white')),
    showlegend=True
)

# Show the figure
fig.show()
